In [1]:
import duckdb
import glob
import pandas as pd
from joblib import Parallel, delayed

# Load Gasperini cis-genes with trans hits
gas_df = pd.read_csv("/gpfs/commons/groups/lappalainen_lab/woliveros/231005_OneK1K/analysis/02_QTL_calling/genes_cis_w_trans.txt", sep='\t', header=None)
gas_cis_genes = set(gas_df[0].tolist())  # Convert to sorted list

# Load morris cis-genes with trans hits
trans_network_john = pd.read_excel("/gpfs/commons/groups/lappalainen_lab/woliveros/231005_OneK1K/data/science.adh7699_table_s4.xlsx", skiprows=2)

morrs_cis_genes = ["GFI1B", "NFE2", "IKZF1", "HHEX", "RUNX1"]

# Load MAGE fine-mapped cis-genes
file_path = "/gpfs/commons/groups/lappalainen_lab/sghatan/stingseq_eqtl_overlap/data/MAGE/MAGE.v1.0.data/QTL_results/eQTL_results/eQTL_finemapping_results/eQTL_finemapping.significantAssociations.MAGE.v1.0.txt.gz"
mage = pd.read_csv(file_path, compression='gzip', sep='\t', low_memory=False)

# Filter for cis-genes
filtered_mage = mage[mage["geneSymbol"].isin(gas_cis_genes.union(morrs_cis_genes))]

In [ ]:
# Directory containing Parquet files
parquet_dir = "/gpfs/commons/groups/lappalainen_lab/sghatan/stingseq_eqtl_overlap/data/MetaLCL/"
parquet_files = glob.glob(parquet_dir + "*.parquet")

# Convert chromosome names safely
filtered_mage = filtered_mage.assign(variantChrom=filtered_mage["variantChrom"].str.replace("chr", "", regex=True))

# Generate WHERE clause for filtering
conditions = " OR ".join(
    [f"(CAST(chromosome AS STRING) = '{row.variantChrom}' AND position = {row.variantPosition})"
     for _, row in filtered_mage.iterrows()]
)

def process_parquet(file):
    """Process a single Parquet file and return filtered results."""
    print(f"Processing file {file}", flush=True)
    try:
        query = f"""
        SELECT * FROM parquet_scan('{file}')
        WHERE {conditions}
        """
        df = duckdb.query(query).to_df()

        if not df.empty:
            df["chromosome"] = df["chromosome"].astype(str)
            filtered_mage["variantChrom"] = filtered_mage["variantChrom"].astype(str)

            # Merge additional columns
            df_merged = df.merge(filtered_mage, left_on=["chromosome", "position"],
                                 right_on=["variantChrom", "variantPosition"], how="left")

            df_merged.drop(columns=["variantChrom", "variantPosition"], inplace=True)

            if "nlog10p" in df_merged.columns and "molecular_trait_id" in df_merged.columns and "geneSymbol" in df_merged.columns:
                df_unique = df_merged.loc[df_merged.groupby(["molecular_trait_id", "geneSymbol"])["nlog10p"].idxmax()]
                df_unique = df_unique.rename(columns={"geneSymbol": "cis_gene"})
                return df_unique

        return None

    except Exception as e:
        print(f"Error processing {file}: {e}")
        return None

# Parallel execution
num_cores = 16  # Adjust this based on available resources
filtered_data = Parallel(n_jobs=num_cores)(delayed(process_parquet)(file) for file in parquet_files)

# Filter out None values
filtered_data = [df for df in filtered_data if df is not None]

# Combine results
if filtered_data:
    final_df = pd.concat(filtered_data, ignore_index=True)
    final_df.to_parquet("MetaLCL_extracted_transQTLs.parquet", index=False)
    print(f"Extracted {len(final_df)} rows and saved to 'MetaLCL_extracted_transQTLs.parquet'.")
else:
    print("No matching rows found in any file.")

Processing file /gpfs/commons/groups/lappalainen_lab/sghatan/stingseq_eqtl_overlap/data/MetaLCL/Liang_2013_ALSPAC_CAP_eur_GENCORD_GEUVADIS_GTEx_TwinsUK_CoLausR2_naive_LCL_metaanalysis_ENSG00000145833_id.parquet
Processing file /gpfs/commons/groups/lappalainen_lab/sghatan/stingseq_eqtl_overlap/data/MetaLCL/Liang_2013_ALSPAC_CAP_eur_GENCORD_GEUVADIS_GTEx_TwinsUK_CoLausR2_naive_LCL_metaanalysis_ENSG00000204666_id.parquet
Processing file /gpfs/commons/groups/lappalainen_lab/sghatan/stingseq_eqtl_overlap/data/MetaLCL/Liang_2013_ALSPAC_CAP_eur_GENCORD_GEUVADIS_GTEx_TwinsUK_CoLausR2_naive_LCL_metaanalysis_ENSG00000187713_id.parquet
Processing file /gpfs/commons/groups/lappalainen_lab/sghatan/stingseq_eqtl_overlap/data/MetaLCL/Liang_2013_ALSPAC_CAP_eur_GENCORD_GEUVADIS_GTEx_TwinsUK_CoLausR2_naive_LCL_metaanalysis_ENSG00000165948_id.parquet
Processing file /gpfs/commons/groups/lappalainen_lab/sghatan/stingseq_eqtl_overlap/data/MetaLCL/Liang_2013_ALSPAC_CAP_eur_GENCORD_GEUVADIS_GTEx_TwinsUK_CoLa

In [2]:
# Query the first 5 rows directly
result = duckdb.sql("SELECT * FROM 'MetaLCL_extracted_transQTLs.parquet' LIMIT 5").df()

# Display result
print(result)

# nlog10p of 1.30103 = 0.05

  chromosome   position ref alt molecular_trait_id      beta        se  \
0          1   26744132   T   C    ENSG00000165948  0.066331  0.063983   
1          3  141911880   A   G    ENSG00000165948 -0.032947  0.021626   
2          5   73512144   G   C    ENSG00000165948  0.092451  0.045834   
3         10  123154480   C   T    ENSG00000165948 -0.030976  0.035349   
4          2   46636733   A   C    ENSG00000165948  0.016676  0.020797   

    nlog10p  n_datasets  sample_size           ac variantRef variantAlt  \
0  0.523071           5         2062   144.687321          T          C   
1  0.894017           7         2926  1729.419344          A          G   
2  1.359703           7         2926   313.619613          G          C   
3  0.419219           7         2926   677.017906          C          T   
4  0.374041           7         2926  2501.307250          A          C   

      variant_kgpID variant_rsID           ensemblID cis_gene  variantPIP  \
0    1:26744132:T:C   rs603

In [5]:
print(filtered_mage.head())

# Count unique rsIDs
unique_rsids_count = filtered_mage["variant_rsID"].nunique()

# Count unique gene symbols
unique_gene_symbols_count = filtered_mage["geneSymbol"].nunique()

print("Number of unique rsIDs:", unique_rsids_count)
print("Number of unique gene symbols:", unique_gene_symbols_count)

     variantChrom  variantPosition variantRef variantAlt   variant_kgpID  \
3682         chr1         26744132          T          C  1:26744132:T:C   
3725         chr1         27914572          G          A  1:27914572:G:A   
4217         chr1         31256672          C          A  1:31256672:C:A   
4220         chr1         31262460          G          A  1:31262460:G:A   
4234         chr1         31284596          G          A  1:31284596:G:A   

     variant_rsID           ensemblID geneSymbol  variantPIP  \
3682   rs60348497  ENSG00000117713.20     ARID1A    1.000000   
3725   rs17185052  ENSG00000117748.10       RPA2    0.962628   
4217   rs13375401  ENSG00000060688.13    SNRNP40    0.028630   
4220  rs149520056  ENSG00000060688.13    SNRNP40    0.028630   
4234  rs116587623  ENSG00000060688.13    SNRNP40    0.028630   

     variantCredibleSet  
3682                 L1  
3725                 L1  
4217                 L1  
4220                 L1  
4234                 L1  
Nu

In [11]:
df = pd.read_parquet('MetaLCL_extracted_transQTLs.parquet')
filtered_df = df[df["nlog10p"] > 11]
unique_gene_symbols_count = filtered_df["cis_gene"].nunique()
#print(filtered_df.head())
print(unique_gene_symbols_count)

26
